In [2]:
# !export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/usr/local/cuda/lib64
!env CUDA_DEVICE_ORDER=PCI_BUS_ID
!env CUDA_VISIBLE_DEVICES=4

LMOD_DIR=/usr/share/lmod/lmod/libexec/
TMUX=/tmp/tmux-22086/default,1030980,0
SSH_CLIENT=103.107.4.31 63325 22
USER=m13518040
LMOD_COLORIZE=yes
MPLBACKEND=module://matplotlib_inline.backend_inline
LMOD_PKG=/usr/share/lmod/lmod
LD_LIBRARY_PATH=/usr/local/cuda/lib:/usr/local/cuda/lib
HOME=/home/m13518040
MOTD_SHOWN=pam
OLDPWD=/raid/data/m13518040
SSH_TTY=/dev/pts/2
PAGER=cat
TF_CPP_MIN_LOG_LEVEL=1
LMOD_sys=Linux
TF2_BEHAVIOR=1
LOGNAME=m13518040
TERM=xterm-color
CUDA_DEVICE_ORDER=PCI_BUS_ID
PATH=/home/m13518040/.local/bin:/usr/local/cuda/bin:/opt/bin/:/usr/local/cuda/bin:/home/m13518040/.local/bin:/usr/local/cuda/bin:/opt/bin/:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/snap/bin
LMOD_FULL_SETTARG_SUPPORT=no
LMOD_PREPEND_BLOCK=normal
KMP_DUPLICATE_LIB_OK=True
KMP_INIT_AT_FORK=FALSE
LANG=en_US.UTF-8
MODULEPATH_ROOT=/sw/modules
SHELL=/bin/sh
LMOD_CMD=/usr/share/lmod/lmod/libexec/lmod
BASH_ENV=/usr/share/lmod/lmod/init/bash
GIT

In [9]:
import os, sys
import gc
import swifter

from sklearn.metrics import accuracy_score
import time

gc.collect()
# os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
# os.environ["CUDA_VISIBLE_DEVICES"] = "4,5"  

import random
import numpy as np
# import jax.numpy as np
import pandas as pd

import torch
from torch import optim
import torch.nn.functional as F
torch.cuda.empty_cache()


import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.corpus import wordnet
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('omw')

stop_words_set = []
for w in stopwords.words('indonesian'):
    stop_words_set.append(w)

import math
import re
import copy

from operator import itemgetter
from deep_translator import GoogleTranslator

from utils.utils_forward_fn import forward_sequence_classification
from utils.utils_init_dataset import set_seed, load_dataset_loader
from utils.utils_semantic_use import USE
from utils.utils_data_utils import DocumentSentimentDataset, DocumentSentimentDataLoader, EmotionDetectionDataset, EmotionDetectionDataLoader
from utils.utils_metrics import document_sentiment_metrics_fn
from utils.utils_init_model import text_logit, fine_tuning_model, eval_model, init_model, logit_prob, load_word_index
from utils.get_args import get_args

# debugger
from icecream import ic

# !pip install pandarallel
from pandarallel import pandarallel
pandarallel.initialize()

from tqdm.notebook import tqdm
tqdm.pandas()
pd.set_option('display.max_colwidth', None)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cuda:3")

INFO: Pandarallel will run on 40 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/m13518040/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/m13518040/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw to /home/m13518040/nltk_data...
[nltk_data]   Package omw is already up-to-date!


In [22]:
# "xlmr-sentiment-codemixing-fr-adv-0.8"
model_target="XLM-R"
downstream_task="sentiment"
attack_strategy="adversarial"
# finetune_epoch=1
num_sample=50
# exp_name=
perturbation_technique="codemixing"
perturb_ratio=0.8
perturb_lang="fr"
seed=26092020
dataset="valid"

set_seed(seed)
use = USE()

In [38]:
tokenizer, config, finetuned_model = init_model(model_target, downstream_task, seed)
w2i, i2w = load_word_index(downstream_task)

train_dataset, train_loader, train_path = load_dataset_loader(downstream_task, 'train', tokenizer)
valid_dataset, valid_loader, valid_path = load_dataset_loader(downstream_task, 'valid', tokenizer)
test_dataset, test_loader, test_path = load_dataset_loader(downstream_task, 'test', tokenizer)

finetuned_model.to(device)
# finetuned_model = fine_tuning_model(model, i2w, train_loader, valid_loader, finetune_epoch)

if dataset == "valid":
    exp_dataset = valid_dataset.load_dataset(valid_path)
elif dataset == "train":
    exp_dataset = train_dataset.load_dataset(train_path)
# exp_dataset = dd.from_pandas(exp_dataset, npartitions=10)
# exp_dataset = te.DataFrame.from_pandas(exp_dataset)
# text,label = None,None

finetuned_model.eval()

torch.set_grad_enabled(False)

LABEL2INDEX = {'positive': 0, 'neutral': 1, 'negative': 2}
total_loss, total_correct, total_labels = 0, 0, 0
list_hyp, list_label = [], []

pbar = tqdm(valid_loader, leave=True, total=len(valid_loader))
for i, batch_data in enumerate(pbar):
    # ic(batch_data)
    _, batch_hyp, _ = forward_sequence_classification(finetuned_model, batch_data[:-1], i2w=i2w, device='cuda')
    list_hyp += batch_hyp
    
# Save prediction
df = pd.DataFrame({'label':list_hyp}).reset_index()
df['label'] = df['label'].apply(lambda sen: LABEL2INDEX[sen])
# df.to_csv('pred.txt', index=False)

print(df)

  0%|          | 0/16 [00:00<?, ?it/s]

      index  label
0         0      2
1         1      2
2         2      0
3         3      0
4         4      2
...     ...    ...
1255   1255      2
1256   1256      2
1257   1257      2
1258   1258      2
1259   1259      0

[1260 rows x 2 columns]


In [16]:
exp_name = "xlmr-sentiment-codemixing-fr-adv-0.8"
# exp_name = "mbert-emotion-sr-adv-0.8-train"
names = exp_name.split("-")

print(exp_name.split("-"))

model_tgt = names[0]
downstream_task = names[1]
seed = 26092020

['xlmr', 'sentiment', 'codemixing', 'fr', 'adv', '0.8']


In [47]:
valid_df = pd.read_csv(os.getcwd() + r'/result/seed'+str(seed)+"/valid/"+exp_name+"-valid"+".csv")

# valid_df["adv_pred"] = df["label"]
valid_df.insert(loc=9, column='adv_pred', value=df["label"].values)
adv_training = accuracy_score(valid_df["sentiment"], valid_df['perturbed_label'])
delta_acc = valid_df.before_attack_acc.values[0] - valid_df.after_attack_acc.values[0]
delta_adv = valid_df.before_attack_acc.values[0] - adv_training 


valid_df.loc[valid_df.index[0], 'adv_training'] = adv_training
valid_df.loc[valid_df.index[0], 'delta_acc'] = delta_acc
valid_df.loc[valid_df.index[0], 'delta_adv'] = delta_adv

valid_df.head()

,text,sentiment,perturbed_text,perturbed_semantic_sim,pred_label,pred_proba,perturbed_label,perturbed_proba,translated_word(s),adv_pred,running_time(s),before_attack_acc,after_attack_acc,avg_semantic_sim,avg_running_time(s),adv_training,delta_acc,delta_adv
0,"meski masa kampanye sudah selesai , bukan berati habis pula upaya mengerek tingkat kedipilihan elektabilitas .",1,"meski masa kampanye sudah selesai , bukan berati habis pula upaya mengerek tingkat kedipilihan elektabilitas .",1.000000,2,[0.0343 0.3033 0.6625],2,[0.0343 0.3033 0.6625],NaN,2,0.05,0.930159,0.876984,0.791034,13.911119,0.876984,0.053175,0.053175
1,tidak enak,2,tidak agréable,0.711760,2,[0.0071 0.0017 0.9912],2,[0.0219 0.0025 0.9756],{'enak': 'agréable'},2,4.36,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"restoran ini menawarkan makanan sunda . kami memesan ayam goreng , kangkung , sayur asam , ikan gurame goreng , ikan bakar , nasi goreng , karedok , tahu tempe , nasi putih , nasi merah etc minuman yang mereka tawarkan juga cukup variatif . rasa makanan enak dan harga murah . kami 9 dewasa dan 5 anak kecil , hanya menghabiskan 800,000",0,restaurant ini offrir aliments Sonde kami commande poulet frire chou frisé sayur asam poisson gurame frire poisson bakar riz frire karedok tahu tempeh riz blanche riz rouge etc boisson yang mereka offrir juga cukup variante rasa aliments agréable dan le prix peu coûteux kami 9 mature dan 5 enfant kecil hanya épuiser,0.588892,0,[0.982 0.0041 0.0138],0,[0.965 0.0153 0.0197],"{'variatif': 'variante', 'harga': 'le prix', 'enak': 'agréable', 'makanan': 'aliments', 'merah': 'rouge', 'murah': 'peu coûteux', 'etc': 'etc', 'minuman': 'boisson', 'goreng': 'frire', 'tempe': 'tempeh', 'nasi': 'riz', 'restoran': 'restaurant', 'ikan': 'poisson', 'memesan': 'commande', 'anak': 'enfant', 'ayam': 'poulet', 'putih': 'blanche', 'tawarkan': 'offrir', 'menghabiskan': 'épuiser', 'menawarkan': 'offrir', 'sunda': 'Sonde', '5': '5', 'dewasa': 'mature', 'kangkung': 'chou frisé'}",0,13.16,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"lokasi di alun alun masakan padang ini cukup terkenal dengan kepala ikan kakap gule , biasa saya pesan nasi bungkus padang berisikan rendang , ayam pop dan perkedel . porsi banyak dan mengenyangkan",0,lokasi di carré carré cuisiner champ ini cukup célèbre dengan kepala poisson vivaneau gule biasa saya un message riz envelopper champ berisikan rendu poulet pop dan Gâteaux portion banyak dan remplissage,0.646173,0,[0.9979 0.001 0.0011],0,[0.9468 0.0361 0.0171],"{'mengenyangkan': 'remplissage', 'perkedel': 'Gâteaux', 'porsi': 'portion', 'pesan': 'un message', 'rendang': 'rendu', 'ayam': 'poulet', 'bungkus': 'envelopper', 'nasi': 'riz', 'pop': 'pop', 'padang': 'champ', 'terkenal': 'célèbre', 'alun': 'carré', 'ikan': 'poisson', 'kakap': 'vivaneau', 'masakan': 'cuisiner'}",0,9.23,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"betapa bejad kader gerindra yang anggota dprd mencabuli anak smp , rakyat harus cerdas partai mana yang harus di tengelamkan di 2019",2,comment dépravé kader gerindra yang membre DPRD agressé enfant lycée gens harus intelligent fête mana yang harus di noyer di 2019,0.686113,2,[0.0044 0.0131 0.9825],1,[0.0135 0.7937 0.1929],"{'tengelamkan': 'noyer', 'mencabuli': 'agressé', 'cerdas': 'intelligent', 'betapa': 'comment', 'anak': 'enfant', 'partai': 'fête', 'rakyat': 'gens', 'anggota': 'membre', 'smp': 'lycée', 'bejad': 'dépravé', 'dprd': 'DPRD'}",2,8.07,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
valid_df.before_attack_acc.values[0]

0.9301587301587302

In [17]:
set_seed(seed)
# use = USE()

#     baca hasil perturb -> finetuning pake perturbed training -> predict data valid
tokenizer, config, model = init_model(model_target, downstream_task, seed)
w2i, i2w = load_word_index(downstream_task)

trainpath = os.getcwd() + r'/result/seed'+str(seed)+"/train/"+exp_name+"-train"+".csv"
validpath = os.getcwd() + r'/result/seed'+str(seed)+"/valid/"+exp_name+"-valid"+".csv"

ic(trainpath)
ic(validpath)

train_dataset, train_loader, _ = load_dataset_loader(downstream_task, 'train', tokenizer, trainpath)
valid_dataset, valid_loader, _ = load_dataset_loader(downstream_task, 'valid', tokenizer, validpath)
# test_dataset, test_loader, test_path = load_dataset_loader(downstream_task, 'test', tokenizer)



if "sentiment" in trainpath: 
    finetuned_model = fine_tuning_model(model, i2w, train_loader, valid_loader, epochs=5)
else: 
    finetuned_model = fine_tuning_model(model, i2w, train_loader, valid_loader, epochs=10)

# valid_df = pd.read_csv(os.getcwd() + r'/result/seed'+str(seed)+"/valid/"+exp_name+"-valid"+".csv")

ic| trainpath: '/raid/data/m13518040/tugas-akhir-repository/result/seed26092020/train/xlmr-sentiment-codemixing-fr-adv-0.8-train.csv'
ic| validpath: '/raid/data/m13518040/tugas-akhir-repository/result/seed26092020/valid/xlmr-sentiment-codemixing-fr-adv-0.8-valid.csv'


TypeError: load_dataset_loader() takes 3 positional arguments but 4 were given

In [ ]:
finetuned_model.eval()

torch.set_grad_enabled(False)

total_loss, total_correct, total_labels = 0, 0, 0
list_hyp, list_label = [], []

pbar = tqdm(valid_loader, leave=True, total=len(test_loader))
for i, batch_data in enumerate(pbar):
    # ic(batch_data)
    _, batch_hyp, _ = forward_sequence_classification(finetuned_model, batch_data[:-1], i2w=i2w, device='cuda')
    list_hyp += batch_hyp

# Save prediction
df = pd.DataFrame({'label':list_hyp}).reset_index()
# df.to_csv('pred.txt', index=False)

print(df)